<img src="../../img/logo_amds.png" alt="Logo" style="width: 128px;"/>

# AmsterdamUMCdb - Freely Accessible ICU Database

version 1.0.2 March 2020  
Copyright &copy; 2003-2020 Amsterdam UMC - Amsterdam Medical Data Science

# Vasopressors and inotropes
Shows medication for artificially increasing blood pressure (vasopressors) or stimulating heart function (inotropes), if any, a patient received.

## Imports

In [1]:
%matplotlib inline
import amsterdamumcdb
import psycopg2
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl

import io
from IPython.display import display, HTML, Markdown

## Display settings

In [2]:
#matplotlib settings for image size
#needs to be in a different cell from %matplotlib inline
plt.style.use('seaborn-darkgrid')
plt.rcParams["figure.dpi"] = 288
plt.rcParams["figure.figsize"] = [16, 12]
plt.rcParams["font.size"] = 12

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = 1000

## Connection settings

In [3]:
#Modify config.ini in the root folder of the repository to change the settings to connect to your postgreSQL database
import configparser
import os
config = configparser.ConfigParser()

if os.path.isfile('../../config.ini'):
    config.read('../../config.ini')
else:
    config.read('../../config.SAMPLE.ini')

#Open a connection to the postgres database:
con = psycopg2.connect(database=config['psycopg2']['database'], 
                       user=config['psycopg2']['username'], password=config['psycopg2']['password'], 
                       host=config['psycopg2']['host'], port=config['psycopg2']['port'])
con.set_client_encoding('WIN1252') #Uses code page for Dutch accented characters.
con.set_session(autocommit=True)

cursor = con.cursor()
cursor.execute('SET SCHEMA \'amsterdamumcdb\''); #set search_path to amsterdamumcdb schema

## Vasopressors and inotropes
from drugitems

In [4]:
sql_vaso_ino = """
WITH vasopressor_inotropes AS (
    SELECT
        admissionid,
        CASE 
            WHEN COUNT(*) > 0 THEN TRUE
            ELSE FALSE
        END AS vasopressors_inotropes_bool,
        STRING_AGG(DISTINCT item, '; ') AS vasopressors_inotropes_given
    FROM drugitems
    WHERE 
        ordercategoryid = 65 -- continuous i.v. perfusor
        AND itemid IN (
            6818, -- Adrenaline (Epinefrine)
            7135, -- Isoprenaline (Isuprel)
            7178, -- Dobutamine (Dobutrex)
            7179, -- Dopamine (Inotropin)
            7196, -- Enoximon (Perfan)
            7229, -- Noradrenaline (Norepinefrine)
            12467, -- Terlipressine (Glypressin)
            13490, -- Methyleenblauw IV (Methylthionide cloride)
            19929 -- Fenylefrine
        )
        AND rate > 0.1
    GROUP BY admissionid
)
SELECT
    a.admissionid, location, 
    CASE 
        WHEN vi.vasopressors_inotropes_bool Then TRUE
        ELSE FALSE
    END AS vasopressors_inotropes_bool,
    vasopressors_inotropes_given
FROM admissions a
LEFT JOIN vasopressor_inotropes vi ON
    a.admissionid = vi.admissionid
"""
vaso_ino = pd.read_sql(sql_vaso_ino,con)
vaso_ino.tail()

,admissionid,location,vasopressors_inotropes_bool,vasopressors_inotropes_given
23101,23548,IC&MC,True,Noradrenaline (Norepinefrine)
23102,23549,MC,True,Noradrenaline (Norepinefrine)
23103,23550,IC,False,None
23104,23551,IC,True,Noradrenaline (Norepinefrine)
23105,23552,MC,False,None
